# The scripts for testing

In [1]:
# Create the stamp for today's date
from datetime import datetime
today = str(datetime.now().strftime("%Y%m%d"))
print(today)

20260225


## Run LD-SDA / LD-BD Benchmarks
This notebook runs the 4 model benchmarks (toy, small-batch, CSTR, column) for both `gdpopt.ldsda` and `gdpopt.ldbd` with a shared configuration, and saves results under `LD_benchmark/results/<YYYYMMDD>/` (summary CSV, traj CSV, solver log).

In [2]:
from pathlib import Path
import sys

# Locate LD_benchmark root and put it on sys.path so we can `import workflow.*`
cwd = Path.cwd().resolve()

def _find_ld_root(start: Path) -> Path:
    if (start / "models").is_dir() and (start / "results").is_dir():
        return start
    if (start / "LD_benchmark" / "models").is_dir():
        return start / "LD_benchmark"
    for parent in [start] + list(start.parents):
        if (parent / "models").is_dir() and (parent / "results").is_dir():
            return parent
        if (parent / "LD_benchmark" / "models").is_dir():
            return parent / "LD_benchmark"
    raise FileNotFoundError(f"Cannot find LD_benchmark root from {start}")

ld_root = _find_ld_root(cwd)
if str(ld_root) not in sys.path:
    sys.path.insert(0, str(ld_root))

# Also add local Pyomo source tree if present (repo layout: <workspace>/pyomo/pyomo/...)
pyomo_src = ld_root.parent / "pyomo"
if (pyomo_src / "pyomo").is_dir() and str(pyomo_src) not in sys.path:
    sys.path.insert(0, str(pyomo_src))

results_day_dir = ld_root / "results" / today
results_day_dir.mkdir(parents=True, exist_ok=True)
print("LD_benchmark root:", ld_root)
print("Pyomo src:", pyomo_src if (pyomo_src / "pyomo").is_dir() else "(not found)")
print("Results dir:", results_day_dir)

LD_benchmark root: D:\Data\Proj\SECQUOIA\Pyomos\LD_benchmark
Pyomo src: D:\Data\Proj\SECQUOIA\Pyomos\pyomo
Results dir: D:\Data\Proj\SECQUOIA\Pyomos\LD_benchmark\results\20260225


In [3]:
from __future__ import annotations
from pathlib import Path
import shutil

from workflow.benchmark_suite import (
    CommonConfig,
    run_toy,
    run_small_batch,
    run_cstr,
    run_column_random_init,
    run_all,
 )

# Common configuration shared by LDSDA and LDBD
common = CommonConfig(
    tee=False,
    time_limit=900,
    direction_norm="Linf",
    nlp_solver="ipopt",
    mip_solver="gurobi",
    separation_solver="gurobi",
    minlp_solver="gams",
 )

# Full solver names (as you clarified)
algorithms = ("gdpopt.ldsda", "gdpopt.ldbd")

# Required CSTR sizes
cstr_NT_list = [5, 10, 15, 20, 25, 30]

# Column random init: set to None to run ALL keys in models/column/column_initial_test.py
# Default here is a small subset to keep the benchmark runtime manageable.
column_keys = list(range(1, 11))

def clear_results(subdir: str | Path) -> Path:
    """Delete `results_day_dir/subdir` if it exists; returns the target path."""
    target = results_day_dir / Path(subdir)
    if target.exists():
        shutil.rmtree(target)
    target.mkdir(parents=True, exist_ok=True)
    return target

print("Configured.")
print("Results day dir:", results_day_dir)
print("Algorithms:", algorithms)

Configured.
Results day dir: D:\Data\Proj\SECQUOIA\Pyomos\LD_benchmark\results\20260225
Algorithms: ('gdpopt.ldsda', 'gdpopt.ldbd')


### Rerun behavior (important)
- `solver.log` and `traj.csv` are **overwritten** for the same test folder when you rerun.
- `summary.csv` is **appended** to (so rerunning the same test will create duplicate rows unless you delete the folder first).
- Column runs write per-init logs under `column/<algo>/init_###/` (overwritten per init), and append to `column/<algo>/summary.csv`.

If you want a completely clean rerun for one benchmark, run `clear_results(...)` for that benchmark folder before rerunning its cell.

In [ ]:
# --- Run 1) Toy problem ---
# Set to True to delete previous toy results for today before rerunning.
CLEAR_TOY_FIRST = False
if CLEAR_TOY_FIRST:
    clear_results("toy")

run_toy(
    date_results_dir=results_day_dir,
    common=common,
    algorithms=algorithms,
 )

print("Toy done.")

In [ ]:
# --- Run 2) Small-batch benchmark ---
CLEAR_SMALL_BATCH_FIRST = False
if CLEAR_SMALL_BATCH_FIRST:
    clear_results("small_batch")

run_small_batch(
    date_results_dir=results_day_dir,
    common=common,
    algorithms=algorithms,
 )

print("Small-batch done.")

In [ ]:
# --- Run 3) CSTR NT sweep ---
CLEAR_CSTR_FIRST = False
if CLEAR_CSTR_FIRST:
    clear_results("cstr")

run_cstr(
    date_results_dir=results_day_dir,
    common=common,
    NT_list=cstr_NT_list,
    algorithms=algorithms,
 )

print("CSTR sweep done.")

In [4]:
# --- Run 4) Column random-initialization benchmark ---
CLEAR_COLUMN_FIRST = False
if CLEAR_COLUMN_FIRST:
    clear_results("column")

run_column_random_init(
    date_results_dir=results_day_dir,
    common=common,
    algorithms=algorithms,
    initial_point_keys=column_keys,
 )

print("Column random-init done.")

WARNING (W1002): Setting Var 'reboil_ratio' to a numeric value `0.9527`
outside the bounds (1.3, 4).
    See also https://pyomo.readthedocs.io/en/latest/errors.html#w1002
WARNING (W1002): Setting Var 'H_L_spec_feed[benzene]' to a numeric value `0`
outside the bounds (0.1, 16).
    See also https://pyomo.readthedocs.io/en/latest/errors.html#w1002
WARNING (W1002): Setting Var 'H_V_spec_feed[benzene]' to a numeric value `0`
outside the bounds (30, 56).
    See also https://pyomo.readthedocs.io/en/latest/errors.html#w1002
WARNING (W1002): Setting Var 'reboil_ratio' to a numeric value `0.9527`
outside the bounds (1.3, 4).
    See also https://pyomo.readthedocs.io/en/latest/errors.html#w1002
WARNING (W1002): Setting Var 'H_L_spec_feed[benzene]' to a numeric value `0`
outside the bounds (0.1, 16).
    See also https://pyomo.readthedocs.io/en/latest/errors.html#w1002
WARNING (W1002): Setting Var 'H_V_spec_feed[benzene]' to a numeric value `0`
outside the bounds (30, 56).
    See also https://p

KeyboardInterrupt: 

In [ ]:
# --- Optional: Run everything ---
# This is convenient for a full benchmark run, but if one test fails you can
# rerun just that specific test cell above.
CLEAR_ALL_FIRST = False
if CLEAR_ALL_FIRST:
    # This wipes today's entire results folder (toy/small_batch/cstr/column)
    # and starts fresh.
    clear_results(".")

run_all(
    date_results_dir=results_day_dir,
    common=common,
    cstr_NT_list=cstr_NT_list,
    column_keys=column_keys,
 )

print("All benchmarks done. Results written under:", results_day_dir)